In [8]:
import torch
import numpy as np
from transformers import PerceiverFeatureExtractor, PerceiverTokenizer, PerceiverForMultimodalAutoencoding

In [14]:
# create multimodal inputs
images = torch.randn((1, 16, 3, 224, 224))
audio = torch.randn((1, 30720, 1))
inputs = dict(image=images, audio=audio, label=torch.zeros((images.shape[0], 700)))

In [ ]:
model = PerceiverForMultimodalAutoencoding.from_pretrained("deepmind/multimodal-perceiver")

In [15]:
# in the Perceiver IO paper, videos are auto-encoded in chunks
# each chunk subsamples different index dimensions of the image and audio modality decoder queries
nchunks = 128
image_chunk_size = np.prod((16, 224, 224)) // nchunks
audio_chunk_size = audio.shape[1] // model.config.samples_per_patch // nchunks
# process the first chunk
chunk_idx = 0
subsampling = {
    "image": torch.arange(image_chunk_size * chunk_idx, image_chunk_size * (chunk_idx + 1)),
    "audio": torch.arange(audio_chunk_size * chunk_idx, audio_chunk_size * (chunk_idx + 1)),
    "label": None,
}

In [ ]:
outputs = model(inputs=inputs, subsampled_output_points=subsampling)

In [35]:
preprocessor = model.perceiver.input_preprocessor

In [48]:
preprocessor.modalities["image"](images)[0].size()

torch.Size([1, 50176, 243])

In [49]:
preprocessor.modalities["audio"](audio)[0].size()

torch.Size([1, 1920, 401])

In [64]:
inputs_prcessed, modality_sizes, inputs_without_pos = preprocessor({"image": images, "audio": audio, "label": torch.zeros((images.shape[0], 700))})